In [8]:
import xarray as xr
import numpy as np
from geopy.geocoders import Nominatim
from huggingface_hub import hf_hub_download
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import requests

In [20]:
def download_data(url, mode):
    if mode == 'train':
        data_folder = "../data/e3sm_train/"
    else:
        data_folder = "../data/e3sm_valid/"

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    
    if os.path.exists(data_folder + os.path.basename(url)):
        print(f"File already exists in {data_folder}. Skipping download.")

    else:
        file_name = os.path.join(data_folder, url.split("/")[-1])
            
        response = requests.get(url)
        
        if response.status_code == 200:
            # file_name = url.split("/")[-1]
            with open(file_name, "wb") as file:
                file.write(response.content)
            print(f"File downloaded and saved to {file_name}")
        else:
            print("Failed to download the file.")

In [21]:
def get_data(training_period, validation_period):
    data_base_url = "https://huggingface.co/datasets/LEAP/ClimSim_low-res/tree/main/train"

    mode = 'train'
    response = requests.get(data_base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        folder_links = soup.find_all('a', class_="col-span-8 flex items-center hover:underline md:col-span-4 lg:col-span-3")

        for link in folder_links:
            href = link.get('href')
            if href[-7:] in validation_period:
                mode = 'valid'
            if href.startswith('/datasets/LEAP/ClimSim_low-res/tree/main/train/') and re.match(r'/datasets/LEAP/ClimSim_low-res/tree/main/train/\d{4}-\d{2}', href):
                subfolder_url = urljoin(data_base_url, href)
                if href[-7:] in training_period or href[-7:] in validation_period: 
                    response_sub = requests.get(subfolder_url)
                    if response_sub.status_code == 200:
                            soup_sub = BeautifulSoup(response_sub.text, 'html.parser')
                            file_links = soup_sub.find_all('a', {'title' : 'Download file'})
                            for file_link in file_links:
                                href_sub = file_link.get('href')
                                if re.match(r'/datasets/LEAP/ClimSim_low-res/resolve/main/train/(\d{4}-\d{2}/[^\s]+)', href_sub):
                                    subfile_url = urljoin(subfolder_url, href_sub)
                                    download_data(subfile_url, mode)

    else:
        print("Failed to fetch the page.")


In [4]:
# Regular Expression Output Function

# Users input a start year-month and end year-month. The function then produces a list regular expressions
# (in the format used in the preprocessing notebook) that can be used to extract files within a particular date range.

def generate_file_regex(start_year_month, end_year_month):
    # Split the start and end year-month strings into integers for processing.
    start_year, start_month = map(int, start_year_month.split('-'))
    end_year, end_month = map(int, end_year_month.split('-'))
    
    # Initialize an empty list to store the generated regular expressions.
    regexps = []

    # Iterate through the years from the start year to the end year (inclusive).
    for year in range(start_year, end_year + 1):
        # Format the year as a 4-digit string, padding with zeros if necessary.
        year_str = str(year).zfill(4)

        # Determine the range of months to consider based on the current year.
        if year == start_year and year == end_year:
            month_range = range(start_month, end_month + 1)
        elif year == start_year:
            month_range = range(start_month, 13)
        elif year == end_year:
            month_range = range(1, end_month + 1)
        else:
            month_range = range(1, 13)

        # Create a pattern for the months within the year, separated by the "|" symbol.
        month_pattern = "|".join([str(month).zfill(2) for month in month_range])

        # Construct the regular expression using the year and the month pattern.
        regex = f'E3SM-MMF.mli.{year_str}-({month_pattern})-*-*.nc'

        # Add the generated regular expression to the list.
        regexps.append(regex)

    # Return the list of generated regular expressions.
    return regexps

In [5]:
# Function to allow user input of geographic and temporal data

# Setting locations to "all" does not filter the data by location

def climsim_geo_temporal_data_finder(locations, training_period, validation_period):
    # Function to coerce location names into latitude and longitude
    def coerce_to_lat_lon(location):
        geolocator = Nominatim(user_agent="geo_locator")
        location_info = geolocator.geocode(location)
        if location_info is not None:
            return location_info.latitude, location_info.longitude
        else:
            return None

    # Download the NetCDF file using Hugging Face's function
    repo_id = "LEAP/ClimSim_low-res"  # Replace with the actual repository ID
    filename = "ClimSim_low-res_grid-info.nc"  # Replace with the actual filename
    local_filepath = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")

    # Load the .nc file
    nc_data = xr.open_dataset(local_filepath)

    # Extract grid latitude and longitude values
    latitude = nc_data["lat"].values
    longitude = nc_data["lon"].values

    # Define a function to find the nearest point
    def find_nearest_lat_lon(user_lat, user_lon, latitude, longitude):
        lat_diff = latitude - user_lat
        lon_diff = longitude - user_lon
        distance = np.sqrt(lat_diff ** 2 + lon_diff ** 2)
        # Find the index of the minimum distance
        idx = np.unravel_index(np.argmin(distance), distance.shape)
        return idx

    # Function to classify user locations
    def classify_user_locations(user_locations, nc_data):
        column_numbers = []
        for user_location in user_locations:
            if isinstance(user_location, str):
                user_lat, user_lon = coerce_to_lat_lon(user_location)
                if user_lat is None or user_lon is None:
                    raise ValueError(f"Invalid coordinates for location: {user_location}")

            elif isinstance(user_location, tuple) and len(user_location) == 2:
                user_lat, user_lon = user_location
            else:
                raise ValueError("Invalid user location format")

            # Find the nearest latitude and longitude point
            idx = find_nearest_lat_lon(user_lat, user_lon, latitude, longitude)

            # Get the corresponding column number
            column_number = nc_data["ncol"][idx[0]].values
            column_numbers.append(column_number)

        return column_numbers

    # Call the classification function with user input locations
    if not locations or "all" in locations:
        all_column_numbers = nc_data["ncol"].values.tolist()
        return all_column_numbers, [], []  # No regex patterns for all data
    else:
        latitude_longitude_list = []

        for location in locations:
            if isinstance(location, (str, tuple)):
                latitude_longitude_list.append(location)
            else:
                raise ValueError("Invalid user location format")

        column_numbers = classify_user_locations(latitude_longitude_list, nc_data)

        unique_column_numbers = list(set(np.ravel(column_numbers)))

        # Use the generate_file_regex function to get date range regexes
        training_regexes = generate_file_regex(training_period[0], training_period[1])
        validation_regexes = generate_file_regex(validation_period[0], validation_period[1])

        return unique_column_numbers, training_regexes, validation_regexes

In [23]:
training_period = ("0001-06", "0004-06")
validation_period = ("0005-01", "0005-02")

get_data(training_period, validation_period)


File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping download.
File already exists in ../data/e3sm_train/. Skipping do

In [6]:
# Code Tester

# Locations include place names, coordinates, duplicates, and closeby cities
locations = ["New York, USA", "Tokyo", "Detroit, MI", "Havana, Cuba", (40.7128, -74.0060), (35.682839, 139.759455), (34.0522, -118.2437), (51.5074, -0.1278), "Manaus, Brazil"]

training_period = ("0001-06", "0004-06")
validation_period = ("0008-01", "0009-06")
result = climsim_geo_temporal_data_finder(locations, training_period, validation_period)
result

([26, 328, 178, 56, 58],
 ['E3SM-MMF.mli.0001-(06|07|08|09|10|11|12)-*-*.nc',
  'E3SM-MMF.mli.0002-(01|02|03|04|05|06|07|08|09|10|11|12)-*-*.nc',
  'E3SM-MMF.mli.0003-(01|02|03|04|05|06|07|08|09|10|11|12)-*-*.nc',
  'E3SM-MMF.mli.0004-(01|02|03|04|05|06)-*-*.nc'],
 ['E3SM-MMF.mli.0008-(01|02|03|04|05|06|07|08|09|10|11|12)-*-*.nc',
  'E3SM-MMF.mli.0009-(01|02|03|04|05|06)-*-*.nc'])